In [1]:
import torch
from torchvision import transforms
from torch.autograd import Variable
from PIL import Image
import cv2
import numpy as np

import sys
sys.path.append('../object_detection')
from image_detector import crop_minAreaRect

In [2]:
names = ['acerolas',
 'apples',
 'apricots',
 'avocados',
 'bananas',
 'blackberries',
 'blueberries',
 'cantaloupes',
 'cherries',
 'coconuts',
 'figs',
 'garbage',
 'grapefruits',
 'grapes',
 'guava',
 'kiwifruit',
 'lemons',
 'limes',
 'mangos',
 'olives',
 'oranges',
 'passionfruit',
 'peaches',
 'pears',
 'pineapples',
 'plums',
 'pomegranates',
 'raspberries',
 'strawberries',
 'tomatoes',
 'watermelons']

In [3]:
loader = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

def image_loader(image):
    """load image, returns cuda tensor"""
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image.cuda()  #assumes that you're using GPU

In [4]:
model_path = "mymodel_finetuning_new_v1.pth"
image_path = "../test_images/ban_app.jpg"
model = torch.load(model_path)

In [25]:
def find_objects(path):
    boundaries = [(np.array([10, 10, 10]), np.array([28, 255, 255])), #yellow
                  (np.array([28, 10, 10]), np.array([80, 255, 255])), #green
                  (np.array([0, 10, 10]), np.array([10, 255, 255]))] #red


    image = cv2.imread(path)
    bordersize = 50
    image = cv2.copyMakeBorder(image, top=bordersize, bottom=bordersize,
                                left=bordersize, right=bordersize,
                                borderType=cv2.BORDER_CONSTANT,
                                value=[255, 255, 255])
    image1 = image.copy()
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    images = []

    for (lower, upper) in boundaries:
        mask = cv2.inRange(hsv, lower, upper)
        res = cv2.bitwise_and(image, image, mask=mask)

        im2, contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE,
                                                    cv2.CHAIN_APPROX_SIMPLE)
        index = []

        for i in range(len(contours)):
            if len(contours[i]) < 50:
                index.append(i)

        contours = np.delete(contours, index)

        for contour in contours:
            rect = cv2.minAreaRect(contour)

            box = cv2.boxPoints(rect)
            box = np.int0(box)
            startX, startY = min(box, key = lambda x: x[0])[0], min(box, key = lambda x: x[1])[1]
            cv2.drawContours(image1, [box], 0, (0, 0, 255), 2)
            
            # display the prediction
            img = crop_minAreaRect(image, rect)
            croped_image = image_loader(Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))
            outputs = torch.nn.Softmax()(model(croped_image)).data.cpu().numpy()[0]
            print(outputs)
            ind = outputs.argsort()[-3:][::-1]
            
            label = ""
            for i in ind:
                label += "  {}: {}%".format(names[i], round(outputs[i]*100,1)) + "\n"
            print(label)
        
            y, dy = startY, 15
            for line in label.split('\n'):
                y += dy
                cv2.putText(image1, line, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)

            img[np.where((img == [0, 0, 0]).all(axis=2))] = [255, 255, 255]
            images.append(img)
            
    cv2.imshow('image', image1)
    cv2.waitKey(0)
    
    return image1

In [ ]:
images = find_objects("../test_images/apple.jpg")

# for i in range(len(images)):
#     cv2.imshow(f"{i}", images[i])
# cv2.waitKey(0)

C:\Users\iatsuk\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[7.59186165e-04 4.50112343e-01 3.26808513e-05 3.36645142e-04
 5.71921095e-03 2.49520435e-05 7.94728840e-06 6.45525142e-05
 1.13572831e-04 1.24890925e-04 9.11979377e-03 1.27475214e-04
 2.54393017e-05 1.25048484e-03 4.92807776e-02 1.53959764e-03
 1.00498600e-02 3.74610275e-02 7.15936348e-02 7.09260255e-03
 2.09497412e-05 3.89611414e-05 1.15620176e-04 3.50741476e-01
 1.88407823e-04 2.24325020e-04 6.10781135e-05 4.04313141e-05
 9.66388252e-05 1.23582315e-04 3.51172057e-03]
  apples: 45.0%
  pears: 35.1%
  mangos: 7.2%

